In [237]:
# Author: Eric Liu
# Date: 06 Oct 2025
# Description: The goal of this project is to identify E-Z Pass Fraud (NJ Court employees misusing their EZ Pass for personal use)

# Dataset type: Tabular
# Number of rows: 3,211 rows
# Number of columns: 17 columns
# Target variable: (Yes/No) (Binary Classification)(Fraud)

In [238]:
# Features
# POSTING DATE
# TRANDSACTION DATE
# TAG/PLATE NUMBER
# AGENCY
# DESCRIPTION
# ENTRY TIME
# ENTRY PLAZA
# ENTRY LANE
# EXIT TIME
# EXIT PLAZA
# EXIT LANE
# VEHICLE TYPE CODE
# AMOUNT
# PREPAID
# PLAN/RATE
# FARE TYPE
# BALANCE

In [239]:
# Note
# The DESCRIPTION feature value for Toll transactions is TOLL. There are other values that will need to be excluded such as:
# Paterson Plank Adj
# Prepaid Payment
# Service Fee

In [240]:
import pandas as pd
from datetime import datetime, date
import holidays
from holidays.countries import US

In [241]:
df = pd.read_csv('Ezpass charges March 2025.csv')
df.head()

,POSTING DATE,TRANSACTION DATE,TAG/PLATE NUMBER,AGENCY,DESCRIPTION,ENTRY TIME,ENTRY PLAZA,ENTRY LANE,EXIT TIME,EXIT PLAZA,EXIT LANE,VEHICLE TYPE CODE,AMOUNT,PREPAID,PLAN/RATE,FARE TYPE,BALANCE
0,3/31/2025,3/30/2025,6618548668,PANYNJ,TOLL,NaN,-,-,20:27:03,GWU,4,-,($108.40),Y,BUSINESS,N,"$2,356.54"
1,3/30/2025,3/30/2025,6618548668,NJTP,TOLL,20:02:24,14,04E,20:11:45,18E,11X,1,($5.54),Y,BUSINESS,N,"$2,467.11"
2,3/30/2025,3/30/2025,6618565606,NJTP,TOLL,13:28:36,12,06E,13:35:45,13A,13X,1,($1.88),Y,BUSINESS,N,"$2,472.65"
3,3/30/2025,3/30/2025,6600800108,GSP,TOLL,NaN,-,-,14:18:56,RAS,12S,1,($2.17),Y,BUSINESS,N,"$2,474.53"
4,3/30/2025,3/30/2025,6600800108,NJTP,TOLL,12:35:52,13,19E,12:46:26,10,06X,1,($3.80),Y,BUSINESS,N,"$2,476.70"


In [242]:
df.shape

(3211, 17)

**We need to remove values that are not "TOLLS" in the description**

In [244]:
(df.DESCRIPTION != "TOLL").any()

True

In [245]:
df = df[df['DESCRIPTION'].isin(["TOLL"])]
df.shape

(3204, 17)

In [246]:
(df.DESCRIPTION != "TOLL").any()

False

In [247]:
def is_weekend(date_string):
    """
    Checks if a given date is a weekend (Saturday or Sunday).
    
    Args:
        date_string: A string given in Date Format: "MM/DD/YYYY"
    
    Returns:
        True if the date is a Saturday or Sunday, False otherwise.
        
    Example:
        >>>date_string = df['TRANSACTION DATE'].iloc[0] #Sunday
        >>>is_weekend(date_string)
        True
    """
    datetime_object = datetime.strptime(date_string, "%m/%d/%Y")
    return datetime_object.weekday() > 4

In [248]:
#is_weekend Test
date_string = df['TRANSACTION DATE'].iloc[0] #Sunday
print(is_weekend(date_string))
date_string = df['TRANSACTION DATE'].iloc[30] #Sunday
print(is_weekend(date_string))

True
False


In [249]:
class NJCourtsHolidays(US):
    """NJ Courts custom holiday calendar.

    Extends UnitedStates holidays with court-specific recess days:
          - Day after Thanksgiving
          - Day after Christmas (only if Dec 25 is Thursday)

    Attributes:
        subdiv (str): Subdivision code ("NJ") by default.
        year (int): Inputted date's year

    Example:
        >>>nj_holidays = NJCourtsHolidays()
        >>>datetime.date(2025, 12, 25) in nj_holidays
        True
        >>>datetime.date(2025, 12, 26) in nj_holidays
        True
        >>>datetime.date(2025, 12, 27) in nj_holidays
        False
        >>>datetime.date(2031, 12, 26) in nj_holidays
        True
    """
    def __init__(self, **kwargs):
        kwargs.setdefault("subdiv", "NJ")
        super().__init__(**kwargs)

    def _populate(self, year):
        super()._populate(year)

        # Day after Thanksgiving (Friday)
        self._add_holiday_1_day_past_4th_thu_of_nov("Court Recess (Day after Thanksgiving)")

        # December 26, but ONLY if Dec 25 is a Thursday
        christmas = date(year, 12, 25)
        if christmas.weekday() == 3:  # Thursday = 3
            self._add_holiday_dec_26("Court Recess (Day after Christmas)")

def HolidayChecker(NJCourtsHolidays_Class, date_string):
    """
    Checks if the provided date is a Holiday defined by NJ Courts

    Args:
        NJCourtsHolidays_Class (class): Instance of a NJCourtsHolidays class
        check_date (datetime.date): A string given in Date Format: "MM/DD/YYYY"

    Returns:
        bool: True if the date is a holiday in NJ Courts, False otherwise.
    Example:
        >>>nj_holidays = NJCourtsHolidays()
        >>>date_string = df['TRANSACTION DATE'].iloc[0] #'3/30/2025'
        >>>HolidayChecker(nj_holidays, date_string)
        False
    """
    check_date = datetime.strptime(date_string, "%m/%d/%Y").date()
    return check_date in NJCourtsHolidays_Class

In [250]:
nj_holidays = NJCourtsHolidays(years=[2025])
for dates2, name in sorted(nj_holidays.items()):
     print(dates2, name)

2025-01-01 New Year's Day
2025-01-20 Martin Luther King Jr. Day
2025-02-12 Lincoln's Birthday
2025-02-17 Presidents Day
2025-04-18 Good Friday
2025-05-26 Memorial Day
2025-06-19 Juneteenth National Independence Day
2025-07-04 Independence Day
2025-09-01 Labor Day
2025-10-13 Columbus Day
2025-11-11 Veterans Day
2025-11-27 Thanksgiving Day
2025-11-28 Court Recess (Day after Thanksgiving)
2025-12-25 Christmas Day
2025-12-26 Court Recess (Day after Christmas)


In [251]:
nj_holidays = NJCourtsHolidays()
date_string = df['TRANSACTION DATE'].iloc[0] #'3/30/2025'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '12/25/2025'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '12/26/2025'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '12/27/2025'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '12/26/2026'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '12/26/2031'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

date_string = '11/28/2025'
print(date_string)
print(HolidayChecker(nj_holidays, date_string))

3/30/2025
False
12/25/2025
True
12/26/2025
True
12/27/2025
False
12/26/2026
False
12/26/2031
True
11/28/2025
True


In [252]:
def Weekday_8am_6pm_Checker(transaction_time_hr, date_string):
    """
    Checks if the provided time is outside business hours and on the weekday

    Args:
        transaction_time_hr (int): A value representing the transaction hour between 0 to 23
        date_string: A string given in Date Format: "MM/DD/YYYY"

    Returns:
        bool: True if transaction_time_hr is outside business hours defined by 8am-6pm and on a weekday, False otherwise.

    Example:
    >>>Weekday_8am_6pm_Checker(time_string_to_int("7:59:59"), "10/17/2025")
    True
    >>>Weekday_8am_6pm_Checker(time_string_to_int("8:00:00"), "10/17/2025")
    False
    """
    start_work_hour = 8 #8am
    end_work_hour = 18 #6pm
    outside_work_hour = transaction_time_hr < start_work_hour or transaction_time_hr >= end_work_hour
    return outside_work_hour and not is_weekend(date_string)

def time_string_to_int(time_string):
    """
    Converts a time string to an int between 0 to 23

    Args:
        time_string (string): A string given in Time Format: "HH:MM:SS"

    Returns:
        bool: True if transaction_time_hr is outside business hours defined by 8am-6pm and on a weekday, False otherwise.

    Example:
    >>>time_string_to_int("09:27:03")
    9
    """
    datetime_object = datetime.strptime(time_string.zfill(8), "%H:%M:%S")
    return datetime_object.hour

In [253]:
print(df['EXIT TIME'].iloc[0])
print(time_string_to_int(df['EXIT TIME'].iloc[0]))

print("7:27:03")
print(time_string_to_int("7:27:03"))

print("09:27:03")
time_string_to_int("09:27:03")

20:27:03
20
7:27:03
7
09:27:03


9

In [254]:
print(Weekday_8am_6pm_Checker(22, "10/11/2025"))
print(Weekday_8am_6pm_Checker(6, "10/10/2025"))
print(Weekday_8am_6pm_Checker(time_string_to_int("18:00:03"), "10/13/2025"))
print(Weekday_8am_6pm_Checker(time_string_to_int("17:59:59"), "10/13/2025"))
print(Weekday_8am_6pm_Checker(time_string_to_int("8:00:00"), "10/17/2025"))
print(Weekday_8am_6pm_Checker(time_string_to_int("7:59:59"), "10/17/2025"))
print(Weekday_8am_6pm_Checker(time_string_to_int("18:00:00"), "10/18/2025"))

False
True
True
False
False
True
False


In [255]:
def Vehicle_Size_Checker(vehicle_class_code):
    vehicle_size_limit = 2
    """
    Checks if the provided vehicle_class_code is larger than allowed
    
    Args:
        vehicle_class_code (int): A value representing the vehicle class code between 1 to 4
    
    Returns:
        bool: True if vehicle_class_code is larger than the defined vehicle size limit, False otherwise or if non numeric.

    Example:
        >>>Vehicle_Size_Checker("2")
        False
        >>>Vehicle_Size_Checker("3")
        True
        >>>Vehicle_Size_Checker("-")
        False
    """
    if vehicle_class_code.isnumeric():
        return int(vehicle_class_code) > vehicle_size_limit
    return False

In [256]:
print(df['VEHICLE TYPE CODE'].iloc[0])
print(Vehicle_Size_Checker(df['VEHICLE TYPE CODE'].iloc[0]))

print(df['VEHICLE TYPE CODE'].iloc[1])
print(Vehicle_Size_Checker(df['VEHICLE TYPE CODE'].iloc[1]))

print("3")
print(Vehicle_Size_Checker("3"))

print("2")
print(Vehicle_Size_Checker("2"))

-
False
1
False
3
True
2
False


In [257]:
def CleanAmount(df):
    """
    Transforms a string with a number to a float ex: "($5.32)" to 5.32, "5.32" to 5.32 or "-" to NaN

    Args:
        df (pandas dataframe): Dataset

    Returns:
         df (pandas dataframe): Dataset

    Example:
    >>>#2, 19, 25, 32, 36, 38, 31, 42, 57, 45, 84
    >>>data_list = ["($2)", "($19)", "$25", "32", "($36)", "38", "($31)", "($42)", "($57)", "($45)", "$84", "-", " "] 
    >>>df_test = pd.DataFrame(data_list, columns=['AMOUNT'])
    >>>CleanAmount(df_test)
    [2.0, 19.0, 25.0, 32.0, 36.0, 38.0, 31.0, 42.0, 57.0, 45.0, 84.0, NaN, NaN] #DataFrame
    """
    
    df['AMOUNT'] = df['AMOUNT'].str.replace(r'[\$()]', '', regex=True)
    df['AMOUNT'] = pd.to_numeric(df['AMOUNT'], errors='coerce')
    return df

def Calculate_Upper_Outlier(df):
    """
    Calculates the upper bound outlier limit defined by Formula: Upper Bound = Q3 + 1.5 * IQR

    Args:
        df (pandas dataframe): Dataset

    Returns:
        float: Upper bound outlier limit defined by Formula: Upper Bound = Q3 + 1.5 * IQR

    Example:
    >>>data_list = [2.0, 19.0, 25.0, 32.0, 36.0, 38.0, 31.0, 42.0, 57.0, 45.0, 84.0, NaN, NaN]
    >>>#Q1=28, Q3=43.5, IQR=15.5, Upper Boundadry= 66.75
    >>>df_test = pd.DataFrame(data_list, columns=['AMOUNT'])
    >>>Calculate_Upper_Outlier(df_test)
    66.75
    """
    amount_col = df['AMOUNT'].dropna()
    Q1 = amount_col.quantile(0.25)
    Q3 = amount_col.quantile(0.75)
    print("Q1:", Q1)
    print("Q3:", Q3)
    IQR = Q3-Q1
    Upper_Bound = Q3 + 1.5*IQR
    print("IQR:", IQR)
    print("Upper_Bound:", Upper_Bound)
    return Upper_Bound
    
def Amount_Checker(amount, outlier_limit):
    """
    Checks if the amount is greater than the outler_limit

    Args:
        amount (float): Transaction Cost
        outlier_limit (float): Upper_Bound outlier limit defined by Calculate_Upper_Outlier()

    Returns:
        bool: True if amount is larger than the outlier limit, False otherwise.
    """
    return amount >= outlier_limit

In [258]:
print("CleanAmount()")
data_list = ["($2)", "($19)", "$25", "32", "($36)", "38", "($31)", "($42)", "($57)", "($45)", "$84", "-", " "] 
df_test = pd.DataFrame(data_list, columns=['AMOUNT'])
df_test = CleanAmount(df_test)
print(df_test)

print("\nCalculate_Upper_Outlier()")
Upper_Outlier = Calculate_Upper_Outlier(df_test)
print(Upper_Outlier)

print("\nAmount_Checker()")
print(Amount_Checker(70.0, Upper_Outlier))
print(Amount_Checker(66.75, Upper_Outlier))
print(Amount_Checker(0, Upper_Outlier))

CleanAmount()
    AMOUNT
0      2.0
1     19.0
2     25.0
3     32.0
4     36.0
5     38.0
6     31.0
7     42.0
8     57.0
9     45.0
10    84.0
11     NaN
12     NaN

Calculate_Upper_Outlier()
Q1: 28.0
Q3: 43.5
IQR: 15.5
Upper_Bound: 66.75
66.75

Amount_Checker()
True
True
False


In [259]:
def Is_Out_Of_State(agency):
    """
    Checks if a person has traveled from NJ to a neighboring state: NY, PA, DE and vice versa. Also flags people using tolls out of state.

    Args:
        agency (string): EZ Pass Agencies such as NJTP, SJ, GSP

    Notes:
        EZ-Pass has two types of toll transactions
        Flat Rate: Common in tunnels/bridges or specific areas where a fixed fee is charged regardless of distance traveled. entry_plaza will have no value here
        Time Rate: Charges depend on entry and exit points. Cost is associated based on distance traveled. entry_plaza will have a value here
    Returns:
        bool: True if person has transactions that involve being in another state, False otherwise.

    Example:
    >>>Is_Out_Of_State("NJTP")
    >>>False
    >>>Is_Out_Of_State("SJ")
    >>>False
    >>>Is_Out_Of_State("GSP")
    >>>False
    >>>Is_Out_Of_State("DRJTBC")
    >>>True
    >>>Is_Out_Of_State("PANYNJ")
    >>>True
    """
    if agency in ["GSP", "SJ", "NJTP"]: #GSP, SJ, and NJTP are NJ agencies
        return False
    return True #DRPA, BCBC, DRJTBC are agencies associated with PA and DE.
        

In [260]:
num = 1
A = df["AGENCY"].iloc[num]
print(A)
Is_Out_Of_State(A)


NJTP


False

In [261]:
df["is_out_of_state"] = df["AGENCY"].apply(Is_Out_Of_State)
df["is_out_of_state"]

0        True
1       False
2       False
3       False
4       False
        ...  
3206    False
3207    False
3208    False
3209    False
3210    False
Name: is_out_of_state, Length: 3204, dtype: bool

In [262]:
df["is_weekend"] = df["TRANSACTION DATE"].apply(is_weekend)
df["is_weekend"]
#Test
#print(df["TRANSACTION DATE"].iloc[26])
#print(df["is_weekend"].iloc[26])

0       True
1       True
2       True
3       True
4       True
        ... 
3206    True
3207    True
3208    True
3209    True
3210    True
Name: is_weekend, Length: 3204, dtype: bool

In [263]:
nj_holidays = NJCourtsHolidays()
print(HolidayChecker(nj_holidays, date_string))

df["is_holiday"] = df["TRANSACTION DATE"].apply(lambda x: HolidayChecker(nj_holidays, x))

True


In [264]:
df["Weekday_8am_6pm"] = df["EXIT TIME"].apply(time_string_to_int)
df["Weekday_8am_6pm"] = df.apply(lambda row: Weekday_8am_6pm_Checker(row['Weekday_8am_6pm'], row["TRANSACTION DATE"]), axis=1)

In [265]:
df["vehicle_type_code>2"] = df["VEHICLE TYPE CODE"].apply(Vehicle_Size_Checker)

In [266]:
df

,POSTING DATE,TRANSACTION DATE,TAG/PLATE NUMBER,AGENCY,DESCRIPTION,ENTRY TIME,ENTRY PLAZA,ENTRY LANE,EXIT TIME,EXIT PLAZA,...,AMOUNT,PREPAID,PLAN/RATE,FARE TYPE,BALANCE,is_out_of_state,is_weekend,is_holiday,Weekday_8am_6pm,vehicle_type_code>2
0,3/31/2025,3/30/2025,6618548668,PANYNJ,TOLL,NaN,-,-,20:27:03,GWU,...,($108.40),Y,BUSINESS,N,"$2,356.54",True,True,False,False,False
1,3/30/2025,3/30/2025,6618548668,NJTP,TOLL,20:02:24,14,04E,20:11:45,18E,...,($5.54),Y,BUSINESS,N,"$2,467.11",False,True,False,False,False
2,3/30/2025,3/30/2025,6618565606,NJTP,TOLL,13:28:36,12,06E,13:35:45,13A,...,($1.88),Y,BUSINESS,N,"$2,472.65",False,True,False,False,False
3,3/30/2025,3/30/2025,6600800108,GSP,TOLL,NaN,-,-,14:18:56,RAS,...,($2.17),Y,BUSINESS,N,"$2,474.53",False,True,False,False,False
4,3/30/2025,3/30/2025,6600800108,NJTP,TOLL,12:35:52,13,19E,12:46:26,10,...,($3.80),Y,BUSINESS,N,"$2,476.70",False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206,3/1/2025,3/1/2025,6618560578,GSP,TOLL,NaN,-,-,9:29:45,UNI,...,($2.17),Y,BUSINESS,N,"$4,699.61",False,True,False,False,False
3207,3/1/2025,3/1/2025,6618688861,GSP,TOLL,NaN,-,-,7:28:59,SPT,...,($2.17),Y,BUSINESS,N,"$4,701.78",False,True,False,False,False
3208,3/1/2025,3/1/2025,6600800108,NJTP,TOLL,0:07:00,13,19E,0:15:14,11,...,($2.55),Y,BUSINESS,N,"$4,749.40",False,True,False,False,False
3209,3/1/2025,3/1/2025,6600800108,GSP,TOLL,NaN,-,-,0:19:07,RAS,...,($2.17),Y,BUSINESS,N,"$4,756.30",False,True,False,False,False


In [267]:
df = CleanAmount(df)

In [268]:
Upper_Outlier = Calculate_Upper_Outlier(df)
#df["amount_outlier"] = df["AMOUNT"] >= Upper_Outlier

Q1: 1.45
Q3: 2.17
IQR: 0.72
Upper_Bound: 3.25


In [269]:
Upper_Outlier = Calculate_Upper_Outlier(df)
Upper_Outlier

Q1: 1.45
Q3: 2.17
IQR: 0.72
Upper_Bound: 3.25


3.25

In [270]:
df

,POSTING DATE,TRANSACTION DATE,TAG/PLATE NUMBER,AGENCY,DESCRIPTION,ENTRY TIME,ENTRY PLAZA,ENTRY LANE,EXIT TIME,EXIT PLAZA,...,AMOUNT,PREPAID,PLAN/RATE,FARE TYPE,BALANCE,is_out_of_state,is_weekend,is_holiday,Weekday_8am_6pm,vehicle_type_code>2
0,3/31/2025,3/30/2025,6618548668,PANYNJ,TOLL,NaN,-,-,20:27:03,GWU,...,108.40,Y,BUSINESS,N,"$2,356.54",True,True,False,False,False
1,3/30/2025,3/30/2025,6618548668,NJTP,TOLL,20:02:24,14,04E,20:11:45,18E,...,5.54,Y,BUSINESS,N,"$2,467.11",False,True,False,False,False
2,3/30/2025,3/30/2025,6618565606,NJTP,TOLL,13:28:36,12,06E,13:35:45,13A,...,1.88,Y,BUSINESS,N,"$2,472.65",False,True,False,False,False
3,3/30/2025,3/30/2025,6600800108,GSP,TOLL,NaN,-,-,14:18:56,RAS,...,2.17,Y,BUSINESS,N,"$2,474.53",False,True,False,False,False
4,3/30/2025,3/30/2025,6600800108,NJTP,TOLL,12:35:52,13,19E,12:46:26,10,...,3.80,Y,BUSINESS,N,"$2,476.70",False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206,3/1/2025,3/1/2025,6618560578,GSP,TOLL,NaN,-,-,9:29:45,UNI,...,2.17,Y,BUSINESS,N,"$4,699.61",False,True,False,False,False
3207,3/1/2025,3/1/2025,6618688861,GSP,TOLL,NaN,-,-,7:28:59,SPT,...,2.17,Y,BUSINESS,N,"$4,701.78",False,True,False,False,False
3208,3/1/2025,3/1/2025,6600800108,NJTP,TOLL,0:07:00,13,19E,0:15:14,11,...,2.55,Y,BUSINESS,N,"$4,749.40",False,True,False,False,False
3209,3/1/2025,3/1/2025,6600800108,GSP,TOLL,NaN,-,-,0:19:07,RAS,...,2.17,Y,BUSINESS,N,"$4,756.30",False,True,False,False,False


In [271]:
df["fraud"] = df[['is_weekend', 'is_holiday', 'Weekday_8am_6pm', 'vehicle_type_code>2', 'is_out_of_state']].any(axis=1)

In [307]:
df

,POSTING DATE,TRANSACTION DATE,TAG/PLATE NUMBER,AGENCY,DESCRIPTION,ENTRY TIME,ENTRY PLAZA,ENTRY LANE,EXIT TIME,EXIT PLAZA,...,PREPAID,PLAN/RATE,FARE TYPE,BALANCE,is_out_of_state,is_weekend,is_holiday,Weekday_8am_6pm,vehicle_type_code>2,fraud
0,3/31/2025,3/30/2025,6618548668,PANYNJ,TOLL,NaN,-,-,20:27:03,GWU,...,Y,BUSINESS,N,"$2,356.54",True,True,False,False,False,True
1,3/30/2025,3/30/2025,6618548668,NJTP,TOLL,20:02:24,14,04E,20:11:45,18E,...,Y,BUSINESS,N,"$2,467.11",False,True,False,False,False,True
2,3/30/2025,3/30/2025,6618565606,NJTP,TOLL,13:28:36,12,06E,13:35:45,13A,...,Y,BUSINESS,N,"$2,472.65",False,True,False,False,False,True
3,3/30/2025,3/30/2025,6600800108,GSP,TOLL,NaN,-,-,14:18:56,RAS,...,Y,BUSINESS,N,"$2,474.53",False,True,False,False,False,True
4,3/30/2025,3/30/2025,6600800108,NJTP,TOLL,12:35:52,13,19E,12:46:26,10,...,Y,BUSINESS,N,"$2,476.70",False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3206,3/1/2025,3/1/2025,6618560578,GSP,TOLL,NaN,-,-,9:29:45,UNI,...,Y,BUSINESS,N,"$4,699.61",False,True,False,False,False,True
3207,3/1/2025,3/1/2025,6618688861,GSP,TOLL,NaN,-,-,7:28:59,SPT,...,Y,BUSINESS,N,"$4,701.78",False,True,False,False,False,True
3208,3/1/2025,3/1/2025,6600800108,NJTP,TOLL,0:07:00,13,19E,0:15:14,11,...,Y,BUSINESS,N,"$4,749.40",False,True,False,False,False,True
3209,3/1/2025,3/1/2025,6600800108,GSP,TOLL,NaN,-,-,0:19:07,RAS,...,Y,BUSINESS,N,"$4,756.30",False,True,False,False,False,True


In [311]:
df.to_csv("fraud2.csv")